In [1]:
## Instalaciones

%pip install torch
%pip install open3d

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
## Dependencias

import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from model import PointnetClassifier, PointNetLoss
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv_comp import save_loss_dict_comp

from compare.model import PointNetCls as compPointNet

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cuda.


In [ ]:
# parametros de ejecución
checkpoint_freq = 20        # cada cuantos epoch guardar el modelo
ROOT_DIR = os.getcwd()
CHECKPOINT_DIR = os.path.join(ROOT_DIR, "checkpoint_comp")

loss_used = 'our'           # 'our' o 'their'

# parametros del dataset
classes = [ModelNetClass.MONITOR, ModelNetClass.TABLE, ModelNetClass.TOILET]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparametros
num_global_feats = 1024     # número de features globales calculadas
epochs = 100
learning_rate = 0.001
reg_weight = 0.001


train_data = ModelNet(classes, DatasetType.TRAIN)
validation_data = ModelNet(classes, DatasetType.VALIDATION)
test_data = ModelNet(classes, DatasetType.TEST)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)
classifier = compPointNet(k = num_classes, feature_transform=False)
if DEVICE == 'cuda': classifier.cuda()

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
criterion = PointNetLoss(alpha=None, gamma=0, reg_weight=reg_weight, size_average=True).to(DEVICE)

In [ ]:
loss_dict = {
    "train": {
        "our_loss": list(),
        "their_loss": list(),
        "acc": list()
    },
    "valid": {
        "our_loss": list(),
        "their_loss": list(),
        "acc": list()
    }
}

for epoch in range(1, epochs + 1):
    print(f"Epoch {epoch}:")
    '''
    Entrenamiento
    '''
    classifier = classifier.train()

    train_batch_our_losses = list()
    train_batch_their_losses = list()
    train_batch_accs = list()
    
    for pcds, labels in train_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)

        # Gradientes en cero
        optimizer.zero_grad()
        
        # Hacer predicciones, calcular pérdida
        # out, _, A = classifier(pcds)
        out, A, _ = classifier(pcds)

        our_loss = criterion(out, labels, A, is_train=True)
        their_loss = F.nll_loss(out, labels)

        with torch.no_grad():
            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas
            correct = pred_choice.eq(labels.data).cpu().sum()
            accuracy = correct.item() / float(pcds.size(0))
                
        # Calcular gradiente y optimizar
        if loss_used == 'our':
            our_loss.backward()
        elif loss_used == 'their':
            their_loss.backward()
        optimizer.step()
    
        # Registramos métricas para el batch
        train_batch_our_losses.append(our_loss.item())
        train_batch_our_losses.append(their_loss.item())
        train_batch_accs.append(accuracy)
        
    epoch_train_our_loss = np.mean(train_batch_our_losses)
    epoch_train_their_loss = np.mean(train_batch_their_losses)
    epoch_train_acc = np.mean(train_batch_accs)
    print("  Train Our Loss:\t\t", epoch_train_our_loss)
    print("  Train Their Loss:\t\t", epoch_train_their_loss)
    print("  Train Acc:\t\t", epoch_train_acc)
    loss_dict["train"]["our_loss"].append(epoch_train_our_loss)
    loss_dict["train"]["their_loss"].append(epoch_train_their_loss)
    loss_dict["train"]["acc"].append(epoch_train_acc)

    '''
    Validación
    '''
    with torch.no_grad():
        classifier = classifier.eval()

        valid_batch_our_losses = list()
        valid_batch_their_losses = list()
        valid_batch_accs = list()
        
        for pcds, labels in validation_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)
            
            # Hacer predicciones
            # out, _, A = classifier(pcds)
            out, A, _ = classifier(pcds)

            our_loss = criterion(out, labels, A, is_train=False)
            their_loss = F.nll_loss(out, labels)

            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas
            correct = pred_choice.eq(labels.data).cpu().sum()
            accuracy = correct.item() / float(pcds.size(0))

            # Registramos métricas para el batch
            valid_batch_our_losses.append(our_loss.item())
            valid_batch_their_losses.append(their_loss.item())
            valid_batch_accs.append(accuracy)
            
    epoch_valid_our_loss = np.mean(valid_batch_our_losses)
    epoch_valid_their_loss = np.mean(valid_batch_their_losses)
    epoch_valid_acc = np.mean(valid_batch_accs)
    print("  Validation Our Loss:\t", epoch_valid_our_loss)
    print("  Validation Their Loss:\t", epoch_valid_their_loss)
    print("  Validation Acc:\t", epoch_valid_acc)
    loss_dict["valid"]["loss"].append(epoch_valid_our_loss)
    loss_dict["valid"]["loss"].append(epoch_valid_their_loss)
    loss_dict["valid"]["acc"].append(epoch_valid_acc)

    if epoch % checkpoint_freq == 0:
        path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(epoch).zfill(4)}.pth")
        torch.save(classifier.state_dict(), path)
    

save_loss_dict_comp(loss_dict, path="loss_dict_comp.csv")

Epoch 1:
  Train Loss:		 0.5008342219026465
  Train Acc:		 0.9786184210526315
  Validation Loss:	 1.64987374516204
  Validation Acc:	 0.45703125
Epoch 2:
  Train Loss:		 0.2804914579579705
  Train Acc:		 0.9695723684210527
  Validation Loss:	 0.47627417370676994
  Validation Acc:	 0.7791666666666667
Epoch 3:
  Train Loss:		 0.26690605831773656
  Train Acc:		 0.9555921052631579
  Validation Loss:	 0.5636608190834522
  Validation Acc:	 0.75
Epoch 4:
  Train Loss:		 0.23266472334140226
  Train Acc:		 0.9606230650154798
  Validation Loss:	 0.729114402551204
  Validation Acc:	 0.7265625
Epoch 5:
  Train Loss:		 0.19525401411872162
  Train Acc:		 0.9581559597523219
  Validation Loss:	 0.891761374194175
  Validation Acc:	 0.6786458333333334
Epoch 6:


KeyboardInterrupt: 